### JST Analysis
Biblioteka została stworzona w celu analizowania i czyszczenia danych pochodzących z arkuszów populacji oraz dochodów Jednostek Samorządu Terytorialnego udostępnianych na stronach rządowych. 

Dochody:
https://www.gov.pl/web/finanse/udzialy-za-2020-r
Populacja:
https://stat.gov.pl/obszary-tematyczne/ludnosc/ludnosc/ludnosc-stan-i-struktura-ludnosci-oraz-ruch-naturalny-w-przekroju-terytorialnym-stan-w-dniu-31-12-2020,6,29.html

#### Instalowanie biblioteki
Bibliotekę można zainstalować pobierając plik .whl z repozytorium oraz wywołując ```pip package_path```. 

In [22]:
!pip install JST_analysis-0.0.3-py3-none-any.whl

Processing c:\users\szant\jst\jst_analysis-0.0.3-py3-none-any.whl


You should consider upgrading via the 'C:\Users\szant\AppData\Local\Programs\Python\Python39-32\python.exe -m pip install --upgrade pip' command.


JST-analysis is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


#### Importowanie i ścieżki

In [23]:
from JST_analysis import (
    load_income_excel,
    save_to_excel,
    compare_incomes,
    load_population_excel,
    population_income_merge,
    calculate_JST,
    calculate_tax_mean,
    calculate_estimates,
)

import pandas as pd  # by wyswietlac dobrze dfy

In [24]:
pd.set_option("expand_frame_repr", False)

In [25]:
gminy_2019 = r"data/2019/20200214_Gminy_za_2019.xlsx"
powiaty_2019 = r"data/2019/20200214_Powiaty_za_2019.xlsx"
NPP_2019 = r"data/2019/20200214_Miasta_NPP_za_2019.xlsx"
wojewodztwa_2019 = r"data/2019/20200214_Wojewodztwa_za_2019.xlsx"

gminy_2020 = r"data/2020/20210215_Gminy_2_za_2020.xlsx"
powiaty_2020 = r"data/2020/20210211_Powiaty_za_2020.xlsx"
NPP_2020 = r"data/2020/20210215_Miasta_NPP_2_za_2020.xlsx"
wojewodztwa_2020 = r"data/2020/20210211_Województwa_za_2020.xlsx"

populacja_2020 = r"data/tabela12.xls"

#### Porównanie dochodów JST
Porówanie dochodów można uzyskać poprzez wczytanie plików zawierających dochód przy wykorzystaniu funkcji ```load_income(file_path, only_important=False)```, gdzie polecam wybranie ```only_important=True```, co spowoduje ładniejsze przedstawienie danych.

In [26]:
gminy_income_2019 = load_income_excel(file_path=gminy_2019, only_important=True)
powiaty_income_2019 = load_income_excel(file_path=powiaty_2019, only_important=True)
NPP_income_2019 = load_income_excel(file_path=NPP_2019, only_important=True)
wojewodztwa_income_2019 = load_income_excel(
    file_path=wojewodztwa_2019, only_important=True
)

gminy_income_2020 = load_income_excel(file_path=gminy_2020, only_important=True)
powiaty_income_2020 = load_income_excel(file_path=powiaty_2020, only_important=True)
NPP_income_2020 = load_income_excel(file_path=NPP_2020, only_important=True)
wojewodztwa_income_2020 = load_income_excel(
    file_path=wojewodztwa_2020, only_important=True
)

Ważnym elementem dataframe'ów tworzonych z wykorzystaniem funkcji pochodzących z biblioteki ```JST_analysis``` jest posiadanie atrybutów pozwalających innym funkcjom rozpoznać typ dataframe'a i zastosować do niego dodatkowe funkcje, jak na przykład zapisanie w formie pliku Excel raportu z integralnością danych, który tworzony jest w atrybucie ```df.attrs["integrity_report"]``` .

In [27]:
print(gminy_income_2019.attrs)

{'JST_type': 'Gminy', 'year': '2019', 'tax_rate': 0.3934, 'type': 'income_base'}


In [28]:
print(gminy_income_2019)

          Kod         Gminy         województwo         powiat  Dochody PIT
0     0201011   bolesławiec        dolnośląskie  bolesławiecki  16772887.92
1     0201022   bolesławiec        dolnośląskie  bolesławiecki   5523594.07
2     0201032      gromadka        dolnośląskie  bolesławiecki   1396512.56
3     0201043  nowogrodziec        dolnośląskie  bolesławiecki   3735946.70
4     0201052    osiecznica        dolnośląskie  bolesławiecki   2381936.29
...       ...           ...                 ...            ...          ...
2406  3218013         dobra  zachodniopomorskie        łobeski    808669.11
2407  3218023         łobez  zachodniopomorskie        łobeski   3659243.54
2408  3218032   radowo małe  zachodniopomorskie        łobeski    674396.97
2409  3218043         resko  zachodniopomorskie        łobeski   2642959.94
2410  3218053     węgorzyno  zachodniopomorskie        łobeski   1494223.68

[2411 rows x 5 columns]


Porównanie dochodów realizowanie jest poprzez funkcję ```compare_income(df1, df2)```, gdzie podanie dataframe'ów o tym samym ```.attrs["JST_type"``` spowoduje stworzenie nowego dataframe'a rozszerzonego o kolumny statystyk porówaniania.

In [29]:
gminy_income_compare_2019_2020 = compare_incomes(
    df1=gminy_income_2019, df2=gminy_income_2020
)

print(gminy_income_compare_2019_2020)

compare_incomes: Found 15 integrity problems - they were skipped, check them by calling .attrs['integrity_report'] on the dataframe created with compare_incomes() function
          Kod         Gminy         województwo         powiat  Dochody w 2019  Dochody w 2020 Różnica 2019/2020[%] Różnica 2019/2020[zł]
0     0201011   bolesławiec        dolnośląskie  bolesławiecki     16772887.92     15915665.39                -5.11            -857222.53
1     0201022   bolesławiec        dolnośląskie  bolesławiecki      5523594.07      5424872.70                -1.79             -98721.37
2     0201032      gromadka        dolnośląskie  bolesławiecki      1396512.56      1405830.64                 0.67               9318.08
3     0201043  nowogrodziec        dolnośląskie  bolesławiecki      3735946.70      3758442.89                 0.60              22496.19
4     0201052    osiecznica        dolnośląskie  bolesławiecki      2381936.29      2299249.90                -3.47             -82686.39


Przykładowy wygląd raportu integralności, który zapisywany jest w atrybucie oraz podczas zapisywania dataframe'u z porównaniem do pliku Excel.

In [43]:
print(gminy_income_compare_2019_2020.attrs["integrity_report"])

          Kod         
         2019     2020
147   0224032  0224033
329   0602062  0602063
392   0608052  0608053
623   1004062  1004063
746   1018042  1018043
852   1210023  1210022
1014  1409062  1409063
1114  1420042  1420043
1121  1420112  1420113
1255  1438052  1438053
1417  1813022  1813023
1880  2602092  2602093
1934  2609032  2609033
2080  3001022  3001023
2122  3007052  3007053


In [30]:
NPP_income_compare_2019_2020 = compare_incomes(df1=NPP_income_2019, df2=NPP_income_2020)

print(NPP_income_compare_2019_2020)

     Kod           NPP         województwo powiat  Dochody w 2019  Dochody w 2020 Różnica 2019/2020[%] Różnica 2019/2020[zł]
0   0261  jelenia góra        dolnośląskie      -    3.553625e+07    3.443777e+07                -3.09           -1098479.27
1   0262       legnica        dolnośląskie      -    4.740745e+07    4.662842e+07                -1.64            -779023.86
2   0264       wrocław        dolnośląskie      -    4.890769e+08    4.843599e+08                -0.96           -4717085.90
3   0265     wałbrzych        dolnośląskie      -    4.339693e+07    4.103938e+07                -5.43           -2357554.96
4   0461     bydgoszcz  kujawsko-pomorskie      -    1.771344e+08    1.706577e+08                -3.66           -6476712.79
..   ...           ...                 ...    ...             ...             ...                  ...                   ...
61  3063        leszno       wielkopolskie      -    3.666786e+07    3.607175e+07                -1.63            -596109.62


In [31]:
print(NPP_income_compare_2019_2020.attrs["integrity_report"])

Empty DataFrame
Columns: []
Index: []


Wywołanie funkcji ```save_to_excel(df, file_path)``` na dowolnym dataframe'ie spowoduje zapisanie go do pliku z określonym rozszerzeniem (WAŻNE: należy podać ścieżkę z rozszerzeniem), w przypadku gdy dataframe posiada atrybut z raportem integralności, będzie on również zapisany w formie osobnego pliku.

In [32]:
save_to_excel(
    df=NPP_income_compare_2019_2020,
    file_path=r"data/reports/powiaty_income_comparison_2019_2020.xlsx",
)

Saved succesfully at data/reports/powiaty_income_comparison_2019_2020.xlsx


#### Łączenie danych dochodów i populacji
Operacje na danych zawierających ludność oraz dochód rozpoczynamy od wykorzystania funkcji ```load_population_excel(file_path)```, która utworzy dataframe z rozbiciem populacji na gminy należące do powiatów, należących do województw.

In [33]:
population = load_population_excel(file_path=populacja_2020)

Loading Excel...
Creating population dictionary...
Cleaning dataframes...
Aggregating populations...


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  6.68it/s]


Następnie należy połączyć go z plikiem dochodów (WAŻNE: ```.attrs["JST_type"]``` musi być "Gminy", by uniknąć problemów z łączeniem), niezależnie od poziomu JST, który chcemy analizować później - jest to podstawa.  

In [34]:
income_population = population_income_merge(
    gmina_income_df=gminy_income_2020, populations_df=population
)

print(income_population)

merge: Found 199 integrity problems - they were skipped, check them by calling .attrs['integrity_report'] on the dataframe created with population_income_merge() function
          Kod         Gminy         województwo         powiat  Dochody PIT  Populacja
0     0201011   bolesławiec        dolnośląskie  bolesławiecki  15915665.39    25823.7
1     0201022   bolesławiec        dolnośląskie  bolesławiecki   5424872.70     9388.8
2     0201032      gromadka        dolnośląskie  bolesławiecki   1405830.64     3441.6
3     0201043  nowogrodziec        dolnośląskie  bolesławiecki   3758442.89     9554.4
4     0201052    osiecznica        dolnośląskie  bolesławiecki   2299249.90     4541.4
...       ...           ...                 ...            ...          ...        ...
2406  3218013         dobra  zachodniopomorskie        łobeski    790912.60     2771.1
2407  3218023         łobez  zachodniopomorskie        łobeski   3475688.61     9034.2
2408  3218032   radowo małe  zachodniopomorski

#### Średni dochód opodatkowany
W celu obliczenia średniego dochodu opodatkowanego dla gmin, należy wywołać funkcję ```calculate_tax_mean(df)```, która spowoduje rozszerzenie obecnego dataframe'a o kolumnę ze średnim dochodem.

In [35]:
income_population_mean = calculate_tax_mean(df=income_population)

print(income_population_mean)

          Kod         Gminy         województwo         powiat  Dochody PIT  Populacja  Średni dochód opodatkowany
0     0201011   bolesławiec        dolnośląskie  bolesławiecki  15915665.39    25823.7                  616.320101
1     0201022   bolesławiec        dolnośląskie  bolesławiecki   5424872.70     9388.8                  577.802563
2     0201032      gromadka        dolnośląskie  bolesławiecki   1405830.64     3441.6                  408.481706
3     0201043  nowogrodziec        dolnośląskie  bolesławiecki   3758442.89     9554.4                  393.372989
4     0201052    osiecznica        dolnośląskie  bolesławiecki   2299249.90     4541.4                  506.286586
...       ...           ...                 ...            ...          ...        ...                         ...
2406  3218013         dobra  zachodniopomorskie        łobeski    790912.60     2771.1                  285.414673
2407  3218023         łobez  zachodniopomorskie        łobeski   3475688.61     

#### Średnia i wariancja
Otrzymujemy ją poprzez wywołanie funkcji ```calculate_JST(df, JST_type)```, gdzie dataframe bazowy dla operacji na ludności/dochodzie, czyli podobnie jak wcześniej - ten utworzony z ```population_income_merge(gmina_income_df, population_df)```.

In [36]:
powiat_statistics = calculate_JST(df=income_population, JST_type="powiat")

print(powiat_statistics)

                Wariancja dochodu opodatkowanego  Średni dochód opodatkowany
powiat                                                                      
aleksandrowski                      1.976154e+12                1.657515e+06
augustowski                         1.798471e+13                2.349241e+06
bartoszycki                         6.336526e+12                2.285536e+06
bełchatowski                        1.041405e+14                6.499756e+06
bialski                             1.493036e+12                1.220329e+06
...                                          ...                         ...
żagański                            9.359239e+12                2.569442e+06
żarski                              1.974467e+13                3.206089e+06
żniński                             4.746912e+12                3.126159e+06
żuromiński                          1.841097e+12                1.489764e+06
żyrardowski                         5.255693e+13                7.357258e+06

In [37]:
wojewodztwo_statistics = calculate_JST(df=income_population, JST_type="województwo")

print(wojewodztwo_statistics)

                     Wariancja dochodu opodatkowanego  Średni dochód opodatkowany
województwo                                                                      
dolnośląskie                             4.334106e+14                2.939891e+07
kujawsko-pomorskie                       2.204236e+14                2.133818e+07
lubelskie                                9.791701e+13                1.915197e+07
lubuskie                                 4.000509e+13                1.949002e+07
mazowieckie                              1.649736e+15                3.902057e+07
małopolskie                              9.158369e+14                4.314544e+07
opolskie                                 1.183996e+14                2.332038e+07
podkarpackie                             1.655835e+14                2.166288e+07
podlaskie                                1.448777e+14                1.316453e+07
pomorskie                                4.371340e+14                3.363892e+07
warmińsko-mazurs

In [38]:
save_to_excel(
    df=powiat_statistics, file_path=r"data/reports/wojewodztwo_statistics.xlsx"
)

Saved succesfully at data/reports/wojewodztwo_statistics.xlsx


#### Estymaty
Estymaty otrzymujemy przez podanie dataframe bazowy dla operacji dochód/populacja, dochodów oraz wybranie poziomu JST dla, którego chcemy je uzyskać.

In [39]:
powiat_estimates = calculate_estimates(
    df_income_population=income_population,
    df_income=powiaty_income_2020,
    JST_level="powiat",
)

print(powiat_estimates)

estimates: Found 18 integrity problems - they were skipped, check them by calling .attrs['integrity_report'] on the dataframe created with calculate_estimates() function
      Kod         Powiaty         województwo  Dochody PIT  Estymata dochodu PIT  Pomyłka Estymata/Dochód
0    0201   bolesławiecki        dolnośląskie   2180844.63          9.035132e+06             6.854287e+06
1    0202  dzierżoniowski        dolnośląskie   2186190.93          7.156400e+06             4.970209e+06
2    0203       głogowski        dolnośląskie   2963921.67          2.529610e+07             2.233218e+07
3    0204        górowski        dolnośląskie    582416.89          3.459999e+06             2.877583e+06
4    0205        jaworski        dolnośląskie   1071205.26          4.662434e+06             3.591229e+06
..    ...             ...                 ...          ...                   ...                      ...
309  3214     stargardzki  zachodniopomorskie   2849690.55          1.647944e+07        

In [40]:
wojewodztwo_estimates = calculate_estimates(
    df_income_population=income_population,
    df_income=wojewodztwa_income_2020,
    JST_level="województwo",
)

print(wojewodztwo_estimates)

estimates: Found 1 integrity problems - they were skipped, check them by calling .attrs['integrity_report'] on the dataframe created with calculate_estimates() function
   Kod          Województwa powiat  Dochody PIT  Estymata dochodu PIT  Pomyłka Estymata/Dochód
0   02         dolnośląskie      -   2300594.34          9.803795e+06             7.503201e+06
1   04   kujawsko-pomorskie      -   1247015.17          5.705343e+06             4.458328e+06
2   06            lubelskie      -   1043651.36          3.878908e+06             2.835257e+06
3   08             lubuskie      -    622331.70          5.401988e+06             4.779657e+06
4   10              łódzkie      -   1710839.86          9.025769e+06             7.314929e+06
5   12          małopolskie      -   2476311.15          7.994579e+06             5.518268e+06
6   14          mazowieckie      -   6083635.79          1.225922e+07             6.175582e+06
7   16             opolskie      -    578928.67          7.407787e+06  

In [41]:
powiat_estimates.attrs

{'integrity_report':       Kod                     Powiaty    województwo powiat_x  Dochody PIT powiat_y   0     _merge
 5    0206  karkonoski (jeleniogórski)   dolnośląskie        -   1583042.90      NaN NaN  left_only
 216  2401                   będziński        śląskie        -   4940722.22      NaN NaN  left_only
 218  2403                  cieszyński        śląskie        -   4955579.49      NaN NaN  left_only
 219  2404               częstochowski        śląskie        -   3393067.44      NaN NaN  left_only
 220  2405                    gliwicki        śląskie        -   3639864.38      NaN NaN  left_only
 221  2406                    kłobucki        śląskie        -   2247392.14      NaN NaN  left_only
 222  2407                 lubliniecki        śląskie        -   1915457.58      NaN NaN  left_only
 223  2408                  mikołowski        śląskie        -   3817797.92      NaN NaN  left_only
 224  2409                  myszkowski        śląskie        -   1894657.15     

In [42]:
save_to_excel(df=powiat_estimates, file_path=r"data/reports/powiat_estimates.xlsx")

Saved succesfully at data/reports/powiat_estimates.xlsx
Integrity check skipped-report saved at data/reports/powiat_estimates_skipped_estimates.xlsx


#### Testowanie
Wszystkie testy można włączyć jednocześnie wywołując z poziomu repozytorium ```python -m unittest```. Poziom wywołania unittest jest ważny z uwagi na możliwy konflikt ścieżek.

#### Profilowanie
Uzyskano przez wywołanie funkcji:
```
python -m cProfile -s tottime tests/main_routine.py > ./examples/profiler.txt
```

Po analizie wyników profilowania można zauważyć, że zdecydowana większość czasu została stracona przez funkcje wczytujące pliki Excelowe z biblioteki pandas. Rozwiązaniem byłoby znalezienie biblioteki specjalizującej się w efektywnym czasowo wczytywaniu i potencjalnie przechowywania lub początkowe przetwarzanie danych w innym formacie plików. W przypadku częstego wykorzystywania biblioteki na tych samych danych można pokusić się o skorzyst 